In [1]:
import requests
import pandas as pd
import json
import time
import pyodbc 
import urllib
import sqlalchemy
import os
import bs4
import html5lib

In [ ]:
##cria conexão com o banco

DRIVER = 'SQL SERVER'
SERVER = 'DESKTOP-2VSV2T8'
DB_NAME = 'Projeto_Steam'

conn = pyodbc.connect(f"""
        DRIVER={{{DRIVER}}};
        SERVER={SERVER};
        DATABASE={DB_NAME};
        Trust_Connection=yes;
                        """)

##cria cursor para execução dos comandos em sql
cursor = conn.cursor()

##cria tabela de aplicativos geral
cursor.execute('DROP TABLE IF EXISTS Jogos CREATE TABLE Jogos (Appid bigint, Nome varchar(1000))')

##comita para salvar as alterações
cursor.commit()


##insere dados dos apps na tabela
##df.to_sql('apps_steam',con=conn,if_exists='replace')


In [ ]:
##valida numero de paginas da api com os jogos
steam_spy = f'https://steamspy.com/api.php?request=all&page={63}'
novo_request = requests.get(steam_spy)



if novo_request.status_code == 200:
    print('true')
else:
     print('false')

In [ ]:
##extrai dados da api

pagination = 1
lista_de_jogos = []

# steam_spy = 'https://steamspy.com/api.php?request=all&page=1'
# novo_request = requests.get(steam_spy)
# df = json.loads(novo_request.text)


while True :
    try:
        steam_spy = f'https://steamspy.com/api.php?request=all&page={pagination}'
        novo_request = requests.get(steam_spy)
        df =  json.loads(novo_request.text) 
        for x in df:
            lista_de_jogos.append((df[x].get('appid'), df[x].get('name'),df[x].get('developer'),df[x].get('publisher'),df[x].get('ccu'),df[x].get('positive'),df[x].get('negative')))
        
        pagination = pagination + 1
    except:
        print('carga finalizada')
        break








In [ ]:
##tranforma em df e renomeia colunas
df = pd.DataFrame(lista_de_jogos)
df.columns = ['appid','nome','desenvolvedor','distribuidora','ccu','positivo','negativo']

df


In [ ]:
##insere dados dos jogos na tabela

for index,row in df.iterrows():
    cursor.execute(f'insert into Jogos  (appid,nome) values(?,?)',row.appid,row.nome)

cursor.commit()


In [ ]:
##le dados de uma tabela
pd.read_sql_query('select * from Jogos',con=conn) 

In [ ]:
##levar para o power bi

In [ ]:
cursor = conn.cursor()
cursor.execute('CREATE TABLE Jogos (Appid bigint, Nome varchar(1000))')
cursor.commit()




In [ ]:
##df[['appid','nome']][0:4] ##selecionando mais de uma coluna
##df.query('appid == 21090')## filtrando um jogo

In [ ]:
import requests
from bs4 import BeautifulSoup

placas = []

# Faz o download do conteúdo da página
url = "https://store.steampowered.com/hwsurvey/videocard/"
response = requests.get(url)
content = response.content

# Cria um objeto BeautifulSoup para analisar o conteúdo HTML
soup = BeautifulSoup(content, "html.parser")

soup.find_all

# # Encontra a div com o id "sub_stats"
# for i in range(1,400,1):

#     div_sub_stats = soup.find_all("div", {"class": f"substats_row row_0"})


#     # Extrai os dados dentro da div
#     data = div_sub_stats.text.strip()

#     # Formata os dados em colunas
#     columns = data.split('\n')
#     columns = [col.strip() for col in columns if col.strip()]
#     placas.append(columns) 

    
# placas
# # Salva os dados em um arquivo de texto
# filename = "dados2.txt"
# with open(filename, "w") as file:
#     max_length = max(len(col) for col in columns)
#     for col in columns:
#         file.write("{:<{}}\n".format(col, max_length))

# print("Os dados foram salvos em", filename)


In [ ]:


# Faz o download do conteúdo da página
url = "https://store.steampowered.com/hwsurvey/videocard/"
response = requests.get(url)
content = response.content

# Cria um objeto BeautifulSoup para analisar o conteúdo HTML
soup = BeautifulSoup(content, "html.parser")

# Encontra todas as divs com a classe "substats_row row_0"
divs = soup.find_all("div", class_="substats_row row_0")

# Extrai os dados de cada div
data_list = []
for div in divs:
    data = div.get_text(separator='|', strip=True)
    data_list.append(data)

# Imprime os dados
for data in data_list:
    print(data)


In [ ]:
meses = {'DEZ':[],'JAN':[],'FEV':[],'MAR':[],'ABR':[],'RESULTADO':[]}

with open('dados.txt','r') as file:
    lines = file.readlines()

    for i in range(1,len(lines),7):
        line = lines[i].rstrip('\n')
        meses['DEZ'].append(line)
    
    for i in range(2,len(lines),7):
        line = lines[i].rstrip('\n')
        meses['JAN'].append(line)

    for i in range(3,len(lines),7):
        line = lines[i].rstrip('\n')
        meses['FEV'].append(line)

    for i in range(4,len(lines),7):
        line = lines[i].rstrip('\n')
        meses['MAR'].append(line)

    for i in range(5,len(lines),7):
        line = lines[i].rstrip('\n')
        meses['ABR'].append(line)

    for i in range(6,len(lines),7):
        line = lines[i].rstrip('\n')
        meses['RESULTADO'].append(line)
meses



In [ ]:
percentuais = []

with open('dados.txt','r') as file:
    lines = file.readlines()

    for i in range(1,len(lines),7):
        line = lines[i].rstrip('\n').strip()
        percentuais.append(line)
    
    for i in range(2,len(lines),7):
        line = lines[i].rstrip('\n').strip()
        percentuais.append(line)

    # for i in range(3,len(lines),7):
    #     line = lines[i].rstrip('\n').strip()
    #     percentuais.append(line)

    # for i in range(4,len(lines),7):
    #     line = lines[i].rstrip('\n').strip()
    #     percentuais.append(line)

    # for i in range(5,len(lines),7):
    #     line = lines[i].rstrip('\n').strip()
    #     percentuais.append(line)

    # for i in range(6,len(lines),7):
    #     line = lines[i].rstrip('\n').strip()
    #     percentuais.append(line)
percentuais


In [ ]:
for lista in placas.values():
    lista.extend(percentuais[:5]) 

placas



In [ ]:
dicionario = {
    'lista1': [],
    'lista2': [],
    'lista3': [],
    'lista4': []
}

itens = ['Item 1', 'Item 2', 'Item 3', 'Item 4', 'Item 5', 'Item 6', 'Item 7', 'Item 8', 'Item 9', 'Item 10','Item 11','Item 12']

itens_por_lista = 4

inicio = 0
fim = itens_por_lista

for lista in dicionario.values():
    lista.extend(itens[inicio:fim])
    inicio = fim
    fim = inicio + itens_por_lista

print(dicionario)


In [ ]:

itens_por_lista = 5

inicio = 0
fim = itens_por_lista


for lista in placas.values():
    lista.extend(percentuais[inicio:fim])
    print(percentuais[inicio:fim])
    # inicio = fim
    # fim = inicio + itens_por_lista

placas

In [ ]:
with open('dados.txt','r') as file:
    lines = file.readlines()

for i in meses:
    for i in range(1,len(lines),7):
        line = lines[i].rstrip('\n')
        placas.append(line)

In [ ]:
cursor.execute('select * from jogos').fetchall()

conn.close()